### 1. Introduction to the Data
In the last mission, we used a data set on U.S. income from the 1994 census; we'll continue using it here. It contains information on marital status, age, type of work, and more. The target column, high_income, indicates a salary less than or equal to 50k per year (0), or more than 50k per year (1).

You can download the data from the [University of California, Irvine's website](http://archive.ics.uci.edu/ml/datasets/Adult).


In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
income = pd.read_csv("income.csv", index_col = False)

In [3]:
income.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### Converting the categorical variables to numeric variables

In [4]:

categorical_cols = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'high_income']

for cat in categorical_cols:
    income[cat] = pd.Categorical(income[cat]).codes

### Calculate Entropy

In [5]:
def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the frequency for each bin of the column
    counts = np.bincount(column)
    
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy


### Calculating Information Gain

In [6]:
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    # Find the median of the column we're splitting
    column = data[split_name]
    median = column.median()
    
    # Make two subsets of the data, based on the median
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    # Return information gain
    return original_entropy - to_subtract

### Determining the Column to Split On

In [7]:
# A list of columns to potentially split income with
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

def find_best_column(data, target_name, columns):
    # target_name is the name of the target variable
    # columns is a list of potential columns to split on
    information_gains = {}
    for col in columns:
        information_gain = calc_information_gain(data,col, target_name)
        #information_gains.append(information_gain)
        information_gains[col] = information_gain   
    
    # return the best column
    best_col = max(information_gains, key=information_gains.get)
    return best_col

In [8]:
income_split = find_best_column(income, "high_income", columns)
income_split

'marital_status'

### Creating a Simple Recursive Algorithm

In [12]:
# We'll use lists to store our labels for nodes (when we find them)
# Lists can be accessed inside our recursive function, whereas integers can't.  
label_1s = []
label_0s = []

def id3(data, target, columns):
    # The pandas.unique method will return a list of all the unique values in a series
    unique_targets = pd.unique(data[target])
    
    if len(unique_targets) == 1:
        
        # append 1 to label_1s or 0 to label_0s, based on what we should label the nod
        if 0 in unique_targets:
             label_0s.append(0)
        elif 1 in unique_targets:
             label_1s.append(1)
        return
    
        
    # Find the best column to split on in our data
    best_column = find_best_column(data, target, columns)
    # Find the median of the column
    column_median = data[best_column].median()
    
    # Create the two splits
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    
    # Loop through the splits and call id3 recursively
    for split in [left_split, right_split]:
        # Call id3 recursively to process each branch
         id3(split, target, columns)

In [13]:
    
# Create the data set that we used in the example on the last screen
data = pd.DataFrame([
    [0,20,0],
    [0,60,2],
    [0,40,1],
    [1,25,1],
    [1,35,2],
    [1,55,1]
    ])
# Assign column names to the data
data.columns = ["high_income", "age", "marital_status"]

# Call the function on our data to set the counters properly
id3(data, "high_income", ["age", "marital_status"])

### Storing the Tree
Now we can store the entire tree, rather than the leaf labels only. We'll use nested dictionaries to do this. We can represent the root node with a dictionary, and branches with the keys left and right. 

In [18]:
# Create a dictionary to hold the tree  
# It has to be outside of the function so we can access it later
tree = {}

# This list will let us number the nodes  
# It has to be a list so we can access it inside the function
nodes = []

def id3(data, target, columns, tree):
    unique_targets = pd.unique(data[target])
    
    # Assign the number key to the node dictionary
    nodes.append(len(nodes) + 1)
    tree["number"] = nodes[-1]  # why -1 , not 0?
   
    if len(unique_targets) == 1:
        # Insert code here that assigns the "label" field to the node dictionary
        if 0 in unique_targets:
            tree['label'] = 0
        elif 1 in unique_targets:
             tree['label'] = 1
                
        return
    
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    # Insert code here that assigns the "column" and "median" fields to the node dictionary
    tree['column'] = best_column
    tree['median'] = column_median
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    split_dict = [["left", left_split], ["right", right_split]]
    
    for name, split in split_dict:
        tree[name] = {}
        id3(split, target, columns, tree[name])

In [19]:
# Call the function on our data to set the counters properly
id3(data, "high_income", ["age", "marital_status"], tree)

In [20]:
tree

{'number': 1,
 'column': 'age',
 'median': 37.5,
 'left': {'number': 2,
  'column': 'age',
  'median': 25.0,
  'left': {'number': 3,
   'column': 'age',
   'median': 22.5,
   'left': {'number': 4, 'label': 0},
   'right': {'number': 5, 'label': 1}},
  'right': {'number': 6, 'label': 1}},
 'right': {'number': 7,
  'column': 'age',
  'median': 55.0,
  'left': {'number': 8,
   'column': 'age',
   'median': 47.5,
   'left': {'number': 9, 'label': 0},
   'right': {'number': 10, 'label': 1}},
  'right': {'number': 11, 'label': 0}}}

### Printing Labels for a More Attractive Tree
The tree dictionary shows all of the relevant information, but it doesn't look very nice. We can fix its appearance by printing it out in a nicer format.

In [21]:
def print_with_depth(string, depth):
    # Add space before a string
    prefix = "    " * depth
    # Print a string, and indent it appropriately
    print("{0}{1}".format(prefix, string))
    

In [22]:
   
def print_node(tree, depth):
    # Check for the presence of "label" in the tree
    if "label" in tree:
        # If found, then this is a leaf, so print it and return
        print_with_depth("Leaf: Label {0}".format(tree["label"]), depth)
        # This is critical -- without it, you'll get infinite recursion
        return
    # Print information about what the node is splitting on
    print_with_depth("{0} > {1}".format(tree["column"], tree["median"]), depth)
    
    # Create a list of tree branches
    branches = [tree["left"], tree["right"]]
        
    # Insert code here to recursively call print_node on each branch
    # Don't forget to increment depth when you pass it in
    for branch in branches:
        print_node(branch, depth)
        depth += 1

print_node(tree, 0)

age > 37.5
age > 25.0
age > 22.5
Leaf: Label 0
    Leaf: Label 1
    Leaf: Label 1
    age > 55.0
    age > 47.5
    Leaf: Label 0
        Leaf: Label 1
        Leaf: Label 0


### Making Predictions With the Printed Tree
Making predictions with such a small tree is fairly straightforward, but what if we want to use the entire income dataframe? We wouldn't be able to eyeball predictions; we'd want an automated way to do this instead.

In [23]:
def predict(tree, row):
    print(row)
    
    if "label" in tree:
        return tree["label"]
    
    column = tree["column"]
    median = tree["median"]
    
    # If it's less than or equal, return the result of predicting on the left branch of the tree
    # If it's greater, return the result of predicting on the right branch of the tree
    if row[tree['column']] <= tree["median"]:
        return predict(tree["left"], row)
    elif row[tree['column']] > tree["median"]:
        return predict(tree["right"], row)


In [24]:
# Print the prediction for the first row in our data
print(predict(tree, data.iloc[0]))

high_income        0
age               20
marital_status     0
Name: 0, dtype: int64
high_income        0
age               20
marital_status     0
Name: 0, dtype: int64
high_income        0
age               20
marital_status     0
Name: 0, dtype: int64
high_income        0
age               20
marital_status     0
Name: 0, dtype: int64
0


### Making Multiple Predictions
Now that we can make a prediction for a single row, we can write a function that makes predictions for multiple rows simultanously.

In [28]:
def batch_predict(tree, df):
    return df.apply(lambda x: predict(tree, x), axis=1)
    pass  

In [29]:
new_data = pd.DataFrame([
    [40,0],
    [20,2],
    [80,1],
    [15,1],
    [27,2],
    [38,1]
    ])
# Assign column names to the data
new_data.columns = ["age", "marital_status"]

predictions = batch_predict(tree, new_data)
predictions

age               40
marital_status     0
Name: 0, dtype: int64
age               40
marital_status     0
Name: 0, dtype: int64
age               40
marital_status     0
Name: 0, dtype: int64
age               40
marital_status     0
Name: 0, dtype: int64
age               20
marital_status     2
Name: 1, dtype: int64
age               20
marital_status     2
Name: 1, dtype: int64
age               20
marital_status     2
Name: 1, dtype: int64
age               20
marital_status     2
Name: 1, dtype: int64
age               80
marital_status     1
Name: 2, dtype: int64
age               80
marital_status     1
Name: 2, dtype: int64
age               80
marital_status     1
Name: 2, dtype: int64
age               15
marital_status     1
Name: 3, dtype: int64
age               15
marital_status     1
Name: 3, dtype: int64
age               15
marital_status     1
Name: 3, dtype: int64
age               15
marital_status     1
Name: 3, dtype: int64
age               27
marital_status     

0    0
1    0
2    0
3    0
4    1
5    0
dtype: int64

### Summary
In this mission, we learned how to create a full decision tree model, print the results, and use the tree to make predictions. We applied a modified version of the ID3 algorithm on a small data set for clarity.